In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
names_text = open("names.txt", "r").read()
words = names_text.splitlines()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [4]:
len(words)

32033

In [7]:
chars = ['.'] + sorted(list(set("".join(words)) - set(['.'])))
ctoi = {c: i for i, c in enumerate(chars)}
itoc = {i: c for i, c in enumerate(chars)}

In [ ]:
CONTEXT_LENGTH = 4
X, Y = [], []
for word in words[:5]:
    # print(word)
    context = [0] * CONTEXT_LENGTH
    for c in word:
        idx = ctoi[c]
        X.append(context)
        Y.append(idx)
        # print("".join(itoc[i] for i in context), "----->", itoc[idx])
        context = context[1:] + [idx]


X = torch.tensor(X)
Y = torch.tensor(Y)